# Lecture 10: Constant-temperature molecular dynamics

## Physics 7810, Spring 2020

## 10.1 - Overview
 
Because Hamiltonian dynamics conserves energy, the natural thermodynamic ensemble for molecular dynamics (MD) simulations is the $NVE$ ensemble. In many cases it is more useful to conduct simulations in other ensembles ($NVT$ or $NPT$), as these ensembles correspond to the typical experimental conditions, and may be more convenient for calculating certain quantities (e.g., free energies and phase diagrams).

Here we will focus on methods for carrying out MD simulations at constant temperature $T$, in the $NVT$ ensemble. Methods for maintaining constant pressure in MD simulations will be discussed later.

Before discussing specific constant-$T$ MD methods, it's useful to consider what we are really trying to simulate when we carry out an MD simulation (or any statistical mechanical simulation for that matter). Quite generally, we can regard an MD simulation of a system of $N$ particles with periodic boundary conditions as a model of a small *subsystem* within a much larger, macroscopic system, in which the remainder of the macroscopic system acts as a thermal reservoir (heat bath) that maintains the subsystem at a constant *average* temperature $T$. The macroscopic reservoir may also exchange volume and particles with the subsystem to maintain constant pressure $P$ and chemical potential $\mu$, but for now we will only consider exchange of *energy* between the subsystem and the reservoir.

As pointed out by Frenkel and Smit (see Section 6.1), the *instantaneous* kinetic temperature $T_K$ of such a hypothetical subsystem isn't really constant, but fluctuates around an average value related to the average kinetic energy per particle by

$$
k_B T = k_B \langle T_K \rangle = m \langle v_\alpha^2 \rangle,
$$

where $m$ is the particle mass and $v_\alpha$ is the $\alpha$th component of its velocity, and the angle brackets denote a canonical ($NVT$) ensemble average. In the canonical ensemble, the relative variance in the instantaneous kinetic temperature $T_K$ is

$$
\frac{\sigma_{T_K}^2}{\langle T_K \rangle^2}
\equiv \frac{\langle T_K^2 \rangle - \langle T_K \rangle^2}{\langle T_K \rangle^2} = \frac{2}{3N}.
$$

Thus, so-called *isokinetic* MD methods that hold the kinetic energy rigorously constant do not correctly simulate the canonical ensemble, although they may yield the correct canonical distribution for properties that only depend on particle coordinates (not momenta). As such isokinetic schemes offer no particular advantage in terms of simplicity or efficiency relative to alternative methods, they will not be discussed further here.

A bewildering variety of constant-temperature MD algorithms have been proposed, but these methods can be grouped into two general categories, based on how the simulated system is coupled to the heat bath.

*Stochastic* methods couple the system to the heat bath through stochastic impulsive forces that act on randomly selected particles at random intervals. These stochastic interactions with the heat bath can be regarded as Monte Carlo moves that transport the system from one constant-energy shell to another. The method proposed by Andersen (the *Andersen thermostat*) is an example of a stochastic thermostat that we'll discuss shortly. *Langevin dynamics* is another stochastic dynamics method that we'll discuss later in the semester.

*Deterministic* methods regulate the temperature by introducing additional 'thermostat' degrees of freedom in an extended Lagrangian formulation. The most commonly used extended Lagrangian method is the so-called *Nose-Hoover thermostat* (and the related Nose-Hoover chain algorithm), which will be described in more detail below. Integration schemes for such extended Lagrangians can be derived via the same operator-splitting methods that we used to derive the velocity Verlet integrator.

A variety of *ad hoc* thermostats also exist, most notably the *Berendsen thermostat*, a simple method that's useful for 'rough and ready' equilibration of simulated systems, but doesn't rigorously sample the canonical distribution function. 

## 10.2 - The Andersen thermostat
 
The constant-temperature MD scheme proposed by Andersen involves stochastic 'collisions' of randomly selected particles at a constant frequency $\nu$. If successive collisions are uncorrelated, then the distribution of time intervals $t$ between two successive collisions is a Poisson distribution,

$$
P(t;\nu) = \nu \exp(-\nu t),
$$

and $P(t;\nu) dt$ is the probability that the next collision will take place in the time interval $[t,t+dt]$. The strength of the coupling to the heat bath is determined by the frequency of stochastic collisions $\nu$.

A constant-temperature MD simulation then consists of the following steps:
1. Starting with an initial set of positions and momenta $\left\{ {\bf r}^N(0), {\bf p}^N(0) \right\}$, integrate the equations of motion for a time $\Delta t$ (note that $\Delta t$ could correspond to a single integration timestep or some fixed number of integration timesteps).
2. Select $N \nu \Delta t$ particles at random to undergo a collision with the heat bath (the probability of selecting a given particle in a time interval $\Delta t$ is $\nu \Delta t$).
3. If particle $i$ has been selected to undergo a collision, its new velocity is drawn from a Maxwell-Boltzmann distribution corresponding to the desired temperature $T$.

Mixing Hamiltonian dynamics with stochastic collisions turns a reversible MD simulation into a Markov process.

It can be shown (see Frenkel and Smit, Section 6.1) that the canonical ensemble phase space distribution is invariant under repeated applications of the Andersen algorithm (it's the limiting distribution for the Markov process).

This implies that the Andersen algorithm generates a canonical distribution of particle positions and momenta.

This method *does not* generate the correct dynamics, however. In fact, the dynamical properties will depend on the collision frequency $\nu$.

Velocity distribution in a Lennard-Jones fluid ($T^\ast = 2.0$, $\rho^\ast = 0.8442$, $N = 108$). The solid line is the Maxwell-Boltzmann distribution, and the symbols are from MD simulations using the Andersen algorithm with collision rates of $\nu = 0.01$ and $\nu = 0.001$:

<img src="images/Frenkel_Fig_6.1.png" alt="Drawing" style="width: 600px;">

Figure from *Understanding Molecular Simulation: From Algorithms to Applications*, by Daan Frenkel and Berend Smit.

Equation of state for the Lennard-Jones fluid ($T^\ast = 2.0$, $N = 108$), comparing results from MD simuations using the Andersen thermostat (open symbols) with results from MC simulations (closed symbols) and an equation of state from the literature (solid line):

<img src="images/Frenkel_Fig_6.2.png" alt="Drawing" style="width: 600px;">

Figure from *Understanding Molecular Simulation: From Algorithms to Applications*, by Daan Frenkel and Berend Smit.

Mean-squared displacement as a function of time for various values of the collision frequency $\nu$ in the Lennard-Jones fluid ($T^\ast = 2.0$, $\rho^\ast = 0.8442$, $N = 108$):

<img src="images/Frenkel_Fig_6.3.png" alt="Drawing" style="width: 600px;">

Figure from *Understanding Molecular Simulation: From Algorithms to Applications*, by Daan Frenkel and Berend Smit.

## 10.3 - The Nose-Hoover thermostat
 
